In the following notebook, we implement three baseline models for conversational recommendation:

1. Random recommendation
2. Most popular recommendation
3. Zero-shot recommendation


### Libraries

In [1]:
import os
import sys
import json
import numpy as np
from typing import List, Dict, Literal, Any
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate

sys.path.append(os.path.abspath(".."))
from scripts.Tools import read_dialogue, get_conversation_by_id 

### Data

In [2]:
dataset: Literal["Books", "Movies", "Sports", "Electronics"] = "Books"


In [3]:
with open(f"../data/{dataset}/item_map.json", "r") as f:
    item_map = json.load(f)

with open("../data/Books/final_data.jsonl", "r") as f:
    final_data = [json.loads(line) for line in f]

In [4]:
items = list(set([item_map[key].lower().strip() for key in item_map]))

In [5]:
item_interactions = {}
for user_data in final_data:
    for user_id in user_data:
        for item_id in user_data[user_id]["history_interaction"]:
            item_name = item_map[item_id].lower().strip()
            if item_name not in item_interactions:
                item_interactions[item_name] = 0
            item_interactions[item_name] += 1

In [6]:
conversation_id = 0

content = read_dialogue(f"../data/{dataset}/Conversation.txt")
conversation = get_conversation_by_id(content, conversation_id)
lines = conversation.strip().split("\n\n")
messages = []

for line in lines:
    if line.startswith("User:"):
        messages.append(HumanMessage(content=line[5:].strip()))
    elif line.startswith("Agent:"):
        messages.append(AIMessage(content=line[6:].strip()))

### Models

In [12]:
def recommend_random(items: List[str], n: int = 5, **kwargs):
    return np.random.choice(items, n, replace=False).tolist()


def recommend_most_popular(item_interactions: Dict[str, int], n: int = 5, **kwargs):
    most_popular_items = sorted(
        item_interactions, key=item_interactions.get, reverse=True
    )[:n]
    return most_popular_items


def recommend_zero_shot(
    messages: List[BaseMessage], n: int = 5, verbose: bool = False, **kwargs
):

    class Recommendation(BaseModel):
        items: Any = Field(
            description="A list of items. Each item must contain a 'reason' and an 'name'"
        )

    system = f"""You are a recommender system. Recommend {n} items based on the user's conversation. Sort the items by descending order of relevance."""

    prompt = ChatPromptTemplate.from_messages(
        [("system", system), ("placeholder", "{messages}")]
    )

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    structured_llm = llm.with_structured_output(Recommendation, include_raw=True)

    zero_shot_structured_llm = prompt | structured_llm
    response = zero_shot_structured_llm.invoke({"messages": messages})

    prompt_tokens = response["raw"].response_metadata["token_usage"]["prompt_tokens"]
    completion_tokens = response["raw"].response_metadata["token_usage"][
        "completion_tokens"
    ]

    cost = 0.150 * prompt_tokens / 1000000 + 0.600 * completion_tokens / 1000000

    if verbose:
        print(f"Prompt Tokens: {prompt_tokens}")
        print(f"Completion Tokens: {completion_tokens}")
        print(f"Cost: ${cost} USD")

    return response["parsed"].items

In [13]:
recommend_random(items)

['disney christmas storybook collection',
 'haven lake',
 'leaving the illusion',
 'the new american bible (with the revised book of psalms and the revised new testament)',
 'an undivided heart (the sister circle series #3)']

In [14]:
recommend_most_popular(item_interactions)

['gone girl',
 'the girl on the train',
 'the pillars of the earth',
 'the old man and the sea',
 'books" />']

In [16]:
recommend_zero_shot(messages[:1], n=5, verbose=False)

[{'name': 'The Hating Game',
  'reason': 'This romantic comedy features a strong enemies-to-lovers dynamic with plenty of drama and witty banter, similar to the engaging storyline you enjoyed.'},
 {'name': 'Red, White & Royal Blue',
  'reason': 'This book combines romance and drama with a unique twist, focusing on the relationship between the First Son of the United States and a British prince.'},
 {'name': 'The Unhoneymooners',
  'reason': 'This story involves unexpected romance and comedic situations, with a strong focus on character dynamics and drama.'},
 {'name': 'Beach Read',
  'reason': 'This novel features two authors with contrasting styles who challenge each other, leading to romantic tension and personal growth amidst the drama.'},
 {'name': 'It Ends with Us',
  'reason': 'This book delves into deeper emotional themes and relationships, providing a mix of drama and romance that keeps readers engaged.'}]